# Cartographie des Musées Parisiens à Travers les Décennies


Le jeu de données utilisé pour cette analyse provient du site data.culture.gouv.fr et contient des informations sur les musées de France, notamment leur localisation, adresse, coordonnées géographiques, et autres détails administratifs. L'URL source est https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin. Ce jeu de données permet d'explorer la richesse culturelle française à travers la répartition géographique et l'évolution temporelle des musées.


L'objectif de cette analyse est de générer une carte des musées parisiens avec une classification en décennies, permettant ainsi de visualiser l'évolution de l'implantation de ces institutions culturelles dans la capitale française. En nous concentrant sur les musées situés à Paris, nous visons à mettre en lumière les tendances et les caractéristiques de leur distribution géographique au fil des ans.


## Méthodologie


La méthodologie utilisée pour cette analyse implique plusieurs étapes clés. Tout d'abord, nous avons sélectionné les données pertinentes en utilisant une requête SQL sur le jeu de données chargé. Cette requête a permis de filtrer les musées situés à Paris et de calculer la décennie d'attribution de l'appellation "musée de France" pour chaque musée. Les résultats ont ensuite été traités avec les bibliothèques Pandas et GeoPandas pour convertir les données géographiques au format approprié. Enfin, nous avons utilisé Plotly Express pour créer une visualisation interactive sous forme de carte, représentant la distribution des musées parisiens classés par décennie.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" SELECT 
  "identifiant_museofile",
  CAST(SUBSTRING(CAST("date_arrete_attribution_appellation" AS VARCHAR), 1, 4) AS INTEGER) - (CAST(SUBSTRING(CAST("date_arrete_attribution_appellation" AS VARCHAR), 1, 4) AS INTEGER) % 10) AS decennie,
  ST_AsText("geolocalisation") AS geom_wkt,
  "nom_officiel_du_musee"
FROM 
  loaded_dataset
WHERE 
  "commune" = 'Paris'
ORDER BY 
  decennie """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 50 lignes


,identifiant_museofile,decennie,geom_wkt,nom_officiel_du_musee
0,M1102,2000,POINT (2.280805 48.855398),maison de Balzac
1,M1104,2000,POINT (2.362856 48.85699),musée Carnavalet - histoire de Paris
2,M1101,2000,POINT (2.29843234195393 48.864603220958784),musée d'art moderne de la ville de Paris
3,M9024,2000,POINT (2.341091 48.840814),musée du service de santé des armées du Val-de...
4,M1115,2000,POINT (2.333457 48.842793),musée Zadkine


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui est particulièrement adaptée pour créer des visualisations interactives de données géospatiales. Elle permet de représenter les données sur une carte interactive, facilitant l'exploration et la compréhension des informations géographiques. Dans ce cas, elle est utilisée pour afficher la répartition des musées parisiens par décennie de création sur une carte.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px

df['geometry'] = gpd.GeoSeries.from_wkt(df['geom_wkt'])
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

gdf['lon'] = gdf.geometry.x
gdf['lat'] = gdf.geometry.y

gdf['decennie'] = gdf['decennie'].astype(str)

color_map = {'2000': '#8B4513',
             '2010': '#D2691E',
             '2020': '#F4A460'}

dataviz = px.scatter_mapbox(
    gdf,
    lat='lat',
    lon='lon',
    color='decennie',
    color_discrete_map=color_map,
    hover_name='nom_officiel_du_musee',
    hover_data={'decennie': True, 'lon': False, 'lat': False},
    zoom=11,
    center={'lat': 48.86, 'lon': 2.35},
    mapbox_style='carto-positron',
    title='Musées parisiens par décennie de création (style feutré)'
)

dataviz.update_layout(
    margin=dict(l=10, r=10, t=40, b=10),
    font=dict(family='Georgia', size=14),
    plot_bgcolor='#F5F5DC',
    paper_bgcolor='#F5F5DC',
    legend=dict(
        title='Décennie',
        orientation='h',
        y=-0.05,
        xanchor='center',
        x=0.5
    )
)

dataviz.for_each_trace(lambda t: t.update(marker=dict(size=10, opacity=0.8)))
dataviz

C:\Users\ami\AppData\Local\Temp\ipykernel_9120\4262964118.py:20: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  dataviz = px.scatter_mapbox(


---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250818_202319.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250818_202319.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSESEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?). L'image statique ne sera pas générée.", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur pour que le script de post-traitement sache qu'il s'agit de Folium
        with open(f"{OUTPUT_HTML_NAME}.is_folium", "w") as f:
            f.write("true")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250818_202319.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250818_202319.png


--> Image Plotly sauvegardée avec succès.
